<a href="https://colab.research.google.com/github/jhagopal/AboutMeSite/blob/main/To_scrape_the_contact_details_of_book_authors_globally_and_their_book_details_too.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I've gathered book information and availability data from the "books.toscrape" website, using the Beautiful Soup library. Additionally, I've created a Python script to display content from multiple pages of the website.

In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os

URL = 'https://books.toscrape.com/'
response = requests.get(URL)

page_contents = response.text

with open('Bookswebpage.html','w') as f:
    f.write(page_contents)

from bs4 import BeautifulSoup
doc = BeautifulSoup(page_contents,'html.parser')

def get_book_titles(doc):
    Book_title_tags = doc.find_all('h3')
    Book_titles = []
    for tags in Book_title_tags:
        Book_titles.append(tags.text)
    return Book_titles

def get_book_price(doc):
    Book_price_tags = doc.find_all('p', class_ = 'price_color')
    Book_price = []
    for tags in Book_price_tags:
        Book_price.append(tags.text.replace('Â',''))
    return Book_price

def get_stock_availability(doc):
    Book_stock_tags = doc.find_all('p', class_ = 'instock availability')
    Book_stock = []
    for tags in Book_stock_tags:
        Book_stock.append(tags.text.strip())
    return Book_stock

def get_book_url(Book_title_tags):
    Book_url = []
    for article in Book_title_tags:
        for link in article.find_all('a', href = True):
            url = link['href']
            links = 'https://books.toscrape.com/' + url
            if links not in Book_url:
                Book_url.append(links)
    return Book_url

def get_doc(url):
    response = requests.get(url)
    doc = BeautifulSoup(response.text,'html.parser')
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(response))
    return doc

def scrape_multiple_pages(n):
    URL = 'https://books.toscrape.com/catalogue/page-'
    titles,prices,stocks_availability,urls = [],[],[],[]

    for page in range(1,n+1):
        doc = get_doc(URL + str(page)+ '.html')
        titles.extend(get_book_titles(doc))
        prices.extend(get_book_price(doc))
        stocks_availability.extend(get_stock_availability(doc))
        urls.extend(get_book_url(doc.find_all('h3')))

    book_dict1 = {
                'TITLE':titles,
                'PRICE':prices,
                'STOCK AVAILABILTY':stocks_availability,
                'URL':urls}
    return pd.DataFrame(book_dict1)

scrape_multiple_pages(5).to_csv('SCB.csv',index = None)


I have collected information, including contact details, about authors from a Medium page using the Python script provided below.

In [ ]:
import json
import requests
from bs4 import BeautifulSoup

url = "https://medium.com/javascript-scene/top-javascript-frameworks-and-topics-to-learn-in-2019-b4142f38df20"
soup = BeautifulSoup(requests.get(url).content, "html.parser")
data = json.loads(soup.select_one('[type="application/ld+json"]').contents[0])

# uncomment this to print all LD+JSON data:
# print(json.dumps(data, indent=4))

print("Author:", data["author"]["name"])
print("URL:", data["author"]["url"])

Author: Eric Elliott
URL: https://medium.com/@_ericelliott


I've obtained contact details for authors from Google Scholar using the Beautiful Soup library.

In [ ]:
from bs4 import BeautifulSoup
import requests, lxml, os

headers = {
    'User-agent':
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

proxies = {
  'http': os.getenv('HTTP_PROXY')
}

html = requests.get('https://scholar.google.com/citations?view_op=view_org&hl=en&org=9834965952280547731', headers=headers, proxies=proxies).text
soup = BeautifulSoup(html, 'lxml')

for result in soup.select('.gs_ai_chpr'):
  name = result.select_one('.gs_ai_name a').text
  link = result.select_one('.gs_ai_name a')['href']
  _id = link
  id_identifer = 'user='
  before_keyword, keyword, after_keyword = _id.partition(id_identifer)
  author_id = after_keyword
  affiliations = result.select_one('.gs_ai_aff').text
  email = result.select_one('.gs_ai_eml').text

  try:
    interests = result.select_one('.gs_ai_one_int').text
  except:
    interests = None

  cited_by = result.select_one('.gs_ai_cby').text.split(' ')[2]


  print(f'{name}\nhttps://scholar.google.com{link}\n{author_id}\n{affiliations}\n{email}\n{interests}\n{cited_by}\n')

Young Jun Park
https://scholar.google.com/citations?hl=en&user=RlANTZEAAAAJ
RlANTZEAAAAJ
Samsung Research, Samsung Electronics
Verified email at samsung.com
None
35262

Jang-hwan Kim
https://scholar.google.com/citations?hl=en&user=SozQwVQAAAAJ
SozQwVQAAAAJ
Samsung Electronics
Verified email at samsung.com
Power Electronics
33580

Jeong-Won Lee
https://scholar.google.com/citations?hl=en&user=D41VK7AAAAAJ
D41VK7AAAAAJ
Samsung Medical Center
Verified email at samsung.com
Gynecologic oncology
27924

Oh Young Bang
https://scholar.google.com/citations?hl=en&user=c9ji0m4AAAAJ
c9ji0m4AAAAJ
Professor of Neurology, Samsung Medical Center
Verified email at samsung.com
Stroke
27459

Eugene Chen
https://scholar.google.com/citations?hl=en&user=7PD1n38AAAAJ
7PD1n38AAAAJ
Samsung, Grandis, Cypress Semiconductor, Motorola, NVE
Verified email at samsung.com
MRAM
27372

Jianzhong (Charlie) Zhang
https://scholar.google.com/citations?hl=en&user=FWVZxvUAAAAJ
FWVZxvUAAAAJ
Samsung
Verified email at samsung.com

I attempted to retrieve contact information for co-authors on Google Scholar by utilizing the author's ID and the SerpApi.

In [ ]:
import requests

# Replace with your SerpApi API key
api_key = "Your API Key"

# Define the parameters
params = {
    "engine": "google_scholar_author",
    "hl": "en",
    "author_id": "m8dFEawAAAAJ",
    "view_op": "list_colleagues",
}

# Define the base URL for the SerpApi endpoint
base_url = "https://serpapi.com/search"

# Include the API key in the parameters
params["api_key"] = api_key

# Send an HTTP GET request to the SerpApi endpoint
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Extract the co-authors' names
    co_authors = data.get("co_authors", [])
    co_author_names = [author.get("name", "") for author in co_authors]

    # Print the co-authors' names
    for name in co_author_names:
        print(name)

else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")

Christoph Benzmüller
Joachim Zahnentferner
Pascal Fontaine
Stephan Merz
Tomer Libal
David Deharbe
Giselle Reis
Joseph Boudou
John Slaney
Dale Miller
Mikheil Rukhaia
Martin Riener
Leon Weber-Genzel
Jan Gorzny
Dmytro Kaidalov
James Chapman
Philip Wadler
Michael Peyton Jones
Manuel MT Chakravarty
Oswaldo Duarte Miranda
JCN de Araujo
Odylio D Aguiar
Annika Kanckos
Carlos HC Ribeiro
Joubert de Castro Lima
Ekaterina Lebedeva
Herman Geuvers
